In [1]:
import pandas as pd
import numpy as np
import numpy.linalg as la
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.base import clone
import timeit
import math
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import warnings

In [2]:
def pca(F, X):
    n, d = X.shape
    mu = np.zeros((d, 1))
    Z = np.zeros((d, F))
    for i in range(d):
        mu[i] = (1. / n) * np.sum(X[:, [i]])
    X = X - mu.T
    U, s, Vt = la.svd(X, False)
    g = s[:F]
    for i in range(F):
        g[i] = 1. / g[i]
    W = Vt[:F]
    Z = np.dot(W.T, np.diag(g))
    return (mu, Z)

def pca_proj(X,mu,Z):
    n, d = X.shape
    X = X - mu.T
    return np.dot(X, Z)

In [3]:
def k_fold(k, model, f, X, y, error_type="mse"):
    n, d = X.shape
    z = np.zeros((k, 1))
    for i in range(k):
        T = list(range(int((i * n) / k), int((n * (i + 1) / k))))
        S = [j for j in range(n) if j not in T]
        curr_model = clone(model)

        training_mu, training_Z = pca(f, X[S])
        training_X = pca_proj(X[S], training_mu, training_Z)

        curr_model.fit(training_X, y[S])

        test_X = pca_proj(X[T], training_mu, training_Z)

        # y[T] will be len(T) by 1
        # X[T] will be len(T) by d
        if error_type == "mse":
            z[i] = (1. / len(T)) * np.sum((y[T] - curr_model.predict(test_X)) ** 2)
        elif error_type == "log_mse":
            z[i] = (1. / len(T)) * np.sum((np.log(y[T] + 1) - np.log(curr_model.predict(test_X) + 1)) ** 2)
    return z

In [4]:
def bootstrapping(B, model, f, X, y, error_type="mse"):
    n, d = X.shape
    z = np.zeros((B, 1))
    for i in range(B):
        u = np.random.choice(n, n, replace=True)
        S = np.unique(u)
        T = np.setdiff1d(np.arange(n), S, assume_unique=True)
        curr_model = clone(model)

        training_mu, training_Z = pca(f, X[u])
        training_X = pca_proj(X[u], training_mu, training_Z)

        curr_model.fit(training_X, y[u])

        test_X = pca_proj(X[T], training_mu, training_Z)

        # y[T] will be len(T) by 1
        # X[T] will be len(T) by d
        # theta_hat will be d by 1
        if error_type == "mse":
            z[i] = (1. / len(T)) * np.sum((y[T] - curr_model.predict(test_X)) ** 2)
        elif error_type == "log_mse":
            z[i] = (1. / len(T)) * np.sum((np.log(y[T] + 1) - np.log(curr_model.predict(test_X) + 1)) ** 2)
    return z

In [5]:
def evaluate_model(model, f, X, y, k=5, B=5, error = "mse"):
    ########################KFOLD###################
    print('Evaluating K-fold with %d folds.' % k)
    start_time = timeit.default_timer()
    k_fold_z = k_fold(k, model, f, X, y, error_type=error)
    elapsed = timeit.default_timer() - start_time
    
    k_fold_mse = np.mean(k_fold_z)
    print('K-fold %s: ' % error, k_fold_mse)
    
    k_fold_rmse = math.sqrt(k_fold_mse)
    print('K-fold Square Root %s: ' % error, k_fold_rmse)

    print("Time elapsed for k-fold: ", elapsed)
    
    print()
    print()
    ###################BOOTSTRAPPING################
    print('Evaluating bootstrapping with %d bootstraps.' % B)
    start_time = timeit.default_timer()
    bootstrapping_z = bootstrapping(B, model, f, X, y)
    elapsed = timeit.default_timer() - start_time
    
    bootstrapping_mse = np.mean(bootstrapping_z)
    print('Bootstrapping Mean Squared Error: ', bootstrapping_mse)
    
    bootstrapping_rmse = math.sqrt(bootstrapping_mse)
    print('Bootstrapping Square Root Mean Squared Error: ', bootstrapping_rmse)

    print("Time elapsed for bootstrapping: ", elapsed)
    
    return (k_fold_z, k_fold_mse, k_fold_rmse, bootstrapping_z, bootstrapping_mse, bootstrapping_rmse)

# Loading and Processing Data

In [6]:
kaggle_train = pd.read_csv("train.csv", header=0)
kaggle_test = pd.read_csv("test.csv", header=0)

kaggle_train_X = kaggle_train.iloc[:,:-1]
kaggle_train_y = kaggle_train.iloc[:,-1:]

print(kaggle_train_X.shape)
print(kaggle_train_y.shape)
print(kaggle_test.shape)

(1460, 80)
(1460, 1)
(1459, 80)


## Joins Kaggle Train and Test data together so we can encode at once.

In [7]:
X_full = kaggle_train_X.append(kaggle_test, ignore_index=True)

## Drops columns that have more than 40% null values.

In [8]:
cols_to_drop = []
data_keys = X_full.keys()
for i, b in enumerate((X_full.isnull().sum() / X_full.shape[0]) > 0.4):
    if b:
        cols_to_drop.append(data_keys[i])

X_full_dropped = X_full.drop(cols_to_drop, axis=1)

In [17]:
cols_to_drop

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

## Runs one hot encoding for categorical columns.

In [9]:
catCols = set(X_full_dropped.select_dtypes(include=['object']))

In [10]:
frames = []
for col in catCols:
    oneHot_encoded = pd.get_dummies(X_full_dropped[col])
    oneHot_encoded = oneHot_encoded.add_prefix(col + '_is_')
    frames.append(oneHot_encoded)

X_full_ohe = X_full_dropped.drop(catCols, axis=1)
X_full_ohe = pd.concat([X_full_ohe, pd.concat(frames, axis=1)], axis=1)

## Splits data back into Kaggle Train and Kaggle Test.

In [11]:
kaggle_train_X_psplit = X_full_ohe.loc[X_full_ohe["Id"].between(1,1460)]
kaggle_test_X_psplit = X_full_ohe.loc[X_full_ohe["Id"].between(1461, 1460+1459)]

## Checks for null values and counts for both rows/columns.

In [12]:
# cols
print(kaggle_train_X_psplit.loc[:, kaggle_train_X_psplit.isnull().any()].shape[1])
print(kaggle_test_X_psplit.loc[:, kaggle_test_X_psplit.isnull().any()].shape[1])

# rows
print(kaggle_train_X_psplit.loc[kaggle_train_X_psplit.isnull().any(axis=1)].shape[0])
print(kaggle_test_X_psplit.loc[kaggle_test_X_psplit.isnull().any(axis=1)].shape[0])

3
11
339
313


## Replaces null values with medians of columns.

In [13]:
kaggle_train_X_processed = kaggle_train_X_psplit.fillna(kaggle_train_X_psplit.median())
kaggle_test_X_processed = kaggle_test_X_psplit.fillna(kaggle_test_X_psplit.median())

## Checks for null values and counts for both rows/columns once more.

In [14]:
# cols
print(kaggle_train_X_processed.loc[:, kaggle_train_X_processed.isnull().any()].shape[1])
print(kaggle_test_X_processed.loc[:, kaggle_test_X_processed.isnull().any()].shape[1])

# rows
print(kaggle_train_X_processed.loc[kaggle_train_X_processed.isnull().any(axis=1)].shape[0])
print(kaggle_test_X_processed.loc[kaggle_test_X_processed.isnull().any(axis=1)].shape[0])

0
0
0
0


## Split Kaggle Training into Train/Test since Kaggle Test has no response variables.

In [15]:
# 80:20 train test ratio
test_size = 0.2
# This function splits the training and target sets into random train and test subsets.
# X_train and X_test are subsets of the training data
# y_train and y_test are subsets the the target data
X_train, X_test, y_train, y_test = train_test_split(kaggle_train_X_processed, kaggle_train_y, test_size=test_size)

# PCA Feature Selection

## Selects 50 features.

In [18]:
F = 50
f = F

##  Runs PCA for 50 features on Kaggle train.

In [ ]:
X_kaggle_train_mu, X_kaggle_train_Z = pca(F, kaggle_train_X_processed.values)
X_kaggle_train_pca = pca_proj(kaggle_train_X_processed.values, X_kaggle_train_mu, X_kaggle_train_Z)

##  Projects PCA for Kaggle Test onto Kaggle Train.

In [ ]:
X_kaggle_test_pca = pca_proj(kaggle_test_X_processed.values, X_kaggle_train_mu, X_kaggle_train_Z)

##  Runs PCA for 50 features on the train split of Kaggle train.

In [ ]:
X_train_mu, X_train_Z = pca(F, X_train.values)
X_train_pca = pca_proj(X_train.values, X_train_mu, X_train_Z)

##  Projects PCA for test split onto train split of Kaggle train.

In [ ]:
X_test_pca = pca_proj(X_test.values, X_train_mu, X_train_Z)

# ADABoost

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
adaBoost = AdaBoostRegressor(n_estimators=70)
ada_k_z, ada_k_mse, ada_k_rmse, ada_b_z, ada_b_mse, ada_b_rmse = evaluate_model(adaBoost, f, kaggle_train_X_processed.values, kaggle_train_y.values.ravel(), k=5, B=5)

adaBoost.fit(X_train_pca, y_train.values.ravel())
adaBoost.score(X_test_pca, y_test.values.ravel())

In [ ]:
#View Predicted values
predicted = adaBoost.predict(X_test_pca)
ada_pred = y_test.copy()
ada_pred['predicted'] = predicted
ada_pred.head()

# XGBoost

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(max_depth=3, learning_rate=0.2, booster='gbtree', n_estimators=70)
xg_k_z, xg_k_mse, xg_k_rmse, xg_b_z, xg_b_mse, xg_b_rmse = evaluate_model(xgb, f, kaggle_train_X_processed.values, kaggle_train_y.values.ravel(), k=5, B=5)

xgb.fit(X_train_pca, y_train)
xgb.score(X_test_pca, y_test.values.ravel())

In [ ]:
predicted = xgb.predict(X_test_pca)
xgb_pred = y_test.copy()
xgb_pred['predicted'] = predicted
xgb_pred.head()

# SVM (SVR)

In [ ]:
from sklearn import svm
svr_model = svm.SVR(kernel="poly", coef0=-2500, gamma="auto")
# coef0 only works with poly and sigmoid kernels
# it just puts that value instead of the column of 1's

# without it, this model breaks for some reason

svr_k_z, svr_k_mse, svr_k_rmse, svr_b_z, svr_b_mse, svr_b_rmse = evaluate_model(svr_model, f, kaggle_train_X_processed.values, kaggle_train_y.values.ravel(), k=5, B=5)

# epsilon, degree
svr_model.fit(X_train_pca, y_train.values.ravel())
svr_model.score(X_test_pca, y_test.values.ravel())

In [ ]:
svr_predicted = svr_model.predict(X_test_pca)
svr_pred = y_test.copy()
svr_pred["predicted"] = svr_predicted
svr_pred.head()

# Scatter Plots of predictions for all 3 models.

In [ ]:
def plotScatter(predicted, name):
    colors = ["r", "b"]
    plt.title(name + " Predicted vs Actual Sale Price")
    plt.xlabel("Actual Sale Price")
    plt.ylabel("Predicted Sale Price")
    red_patch = mpatches.Patch(color='red', label='Actual Sale Price')
    blue_patch = mpatches.Patch(color='blue', label='Predicted Sale Price')
    plt.scatter(predicted['SalePrice'], predicted['predicted'], color="r", alpha=0.5)
    plt.show()

In [ ]:
print()
print("========== Prediction Scatter Plots Across Models ==========")
print()

plotScatter(ada_pred, "AdaBoost")
plotScatter(xgb_pred, "XgBoost")
plotScatter(svr_pred, "SVM")

# Kaggle Submission

In [ ]:
kaggle_xgb = XGBRegressor(max_depth=3, learning_rate=0.2, booster='gbtree', n_estimators=70)

kaggle_xgb.fit(X_kaggle_train_pca, kaggle_train_y)

In [ ]:
kaggle_predicted = kaggle_xgb.predict(X_kaggle_test_pca)

In [ ]:
kaggle_predicted_complete = pd.DataFrame({'Id': kaggle_test["Id"], 'SalePrice': kaggle_predicted})

In [ ]:
kaggle_predicted_complete.to_csv('kaggle_predicted.csv', index=False)

# Tuning Hyperparameters

## Tunes AdaBoost/XGBoost n_estimators and SVR's C parameters.

In [ ]:
#This takes time (~2mins)
def tune_hyperparameters(X, Y, f):
    adaboost_param_tuning = pd.DataFrame(columns=['parameter', 'rmse'])
    xgb_param_tuning = pd.DataFrame(columns=['parameter', 'rmse'])    
    svr_param_tuning = pd.DataFrame(columns=['parameter', 'rmse'])    

    #Tuning n estimators parameter for boosting algorithms
    for i in range(25,200,25):
        print("Boosting: " + str(i))
        print()
        adaBoost = AdaBoostRegressor(n_estimators=i)
        print(adaBoost)
        k_fold_z, k_fold_mse, k_fold_rmse, bootstrapping_z, bootstrapping_mse, bootstrapping_rmse = evaluate_model(adaBoost, f, X, Y.values.ravel(), k=5, B=5)
        adaboost_param_tuning = adaboost_param_tuning.append({'parameter': i, 'rmse': k_fold_rmse}, ignore_index=True)
        print()
        
        xgb = XGBRegressor(max_depth=3, learning_rate=0.2, booster='gbtree', n_estimators=i)
        print(xgb)
        k_fold_z, k_fold_mse, k_fold_rmse, bootstrapping_z, bootstrapping_mse, bootstrapping_rmse = evaluate_model(xgb, f, X, Y.values.ravel(), k=5, B=5)
        xgb_param_tuning = xgb_param_tuning.append({'parameter': i, 'rmse': k_fold_rmse}, ignore_index=True)
        print()
        
    #for i in range(25,200,25):
    c_vals = [0.01, 0.1, 10, 100]
    for i in c_vals:
        print("C: " + str(i))
        print()
        svr_model = svm.SVR(kernel="poly", coef0=-2500, gamma="auto", C=i)
        print(svr_model)
        k_fold_z, k_fold_mse, k_fold_rmse, bootstrapping_z, bootstrapping_mse, bootstrapping_rmse = evaluate_model(svr_model, f, X, Y.values.ravel(), k=5, B=5)
        svr_param_tuning = svr_param_tuning.append({'parameter': i, 'rmse': k_fold_rmse}, ignore_index=True)
        print()

    return xgb_param_tuning, adaboost_param_tuning, svr_param_tuning

In [ ]:
print()
print("========== Hyper-Parameter Tuning ==========")
print()

xgb_params, adaboost_params, svm_params = tune_hyperparameters(kaggle_train_X_processed.values, kaggle_train_y, f)

print()
print("*** Hyper-parameter Tuning Complete! ***")
print()

### Plots how the changing of hyperparameters changes our models' root mean squared log errors.

In [ ]:
def plot_param_tuning(xgb_params, adaboost_params, svm_params):
    plt.plot(adaboost_params['parameter'], adaboost_params['rmse'], marker='o', color='b')
    plt.title("Adaboost RMSE vs n_estimators")
    plt.xlabel("n_estimators")
    plt.ylabel("RMSE")
    plt.show()
    
    plt.plot(xgb_params['parameter'], xgb_params['rmse'], marker='o', color='b')
    plt.title("XgBoost RMSE vs n_estimators")
    plt.xlabel("n_estimators")
    plt.ylabel("RMSE")
    plt.show()
    
    plt.plot(svm_params['parameter'], svm_params['rmse'], marker='o', color='b')
    plt.title("SVM RMSE vs C")
    plt.xlabel("C")
    plt.ylabel("RMSE")
    plt.show()

In [ ]:
print()
print("========== Parameter Tuning Plots ==========")
print()

plot_param_tuning(xgb_params, adaboost_params, svm_params)

## Tunes how many features to use from PCA.

In [ ]:
# This takes time to run (~5mins)
def pca_tuning(F, X, Y):
    
    pca_ada_frame = pd.DataFrame(columns=['pca', 'rmse'])
    pca_xg_frame = pd.DataFrame(columns=['pca', 'rmse'])
    pca_svr_frame = pd.DataFrame(columns=['pca', 'rmse'])
    
    for f in F:
        print("\nPCA %d" % f)

        # AdaBoost
        print("\n")
        from sklearn.ensemble import AdaBoostRegressor
        adaBoost = AdaBoostRegressor()
        k_z, k_mse, k_rmse, b_z, b_mse, b_rmse = evaluate_model(adaBoost, f, X.values, Y.values.ravel(), k=5, B=5)
        pca_ada_frame = pca_ada_frame.append({'pca': f, 'rmse': k_rmse}, ignore_index=True)


        # XGBoost Regressor
        print("\nXGBoost")
        from xgboost import XGBRegressor

        xgb = XGBRegressor(max_depth=3, learning_rate=0.2, booster='gbtree', n_estimators=70)

        k_z, k_mse, k_rmse, b_z, b_mse, b_rmse = evaluate_model(xgb, f, X.values, Y.values.ravel(), k=5, B=5)
        pca_xg_frame = pca_xg_frame.append({'pca': f, 'rmse': k_rmse}, ignore_index=True)

        
        # SVM (SVR)
        print("\nSVR")
        from sklearn import svm

        svr_model = svm.SVR(kernel="poly", coef0=-3500, gamma='auto')
        # coef0 only works with poly and sigmoid kernels
        # it just puts that value instead of the column of 1's

        # without it, this model breaks for some reason

        k_z, k_mse, k_rmse, b_z, b_mse, b_rmse = evaluate_model(svr_model, f, X.values, Y.values.ravel(), k=5, B=5)
        pca_svr_frame = pca_svr_frame.append({'pca': f, 'rmse': k_rmse}, ignore_index=True)
        
    return (pca_ada_frame, pca_xg_frame, pca_svr_frame)

### Plots how different amounts of PCA features affects our models' performances.

In [ ]:
def plot_pca_tuning(ada_data, xg_data, svr_data):
    plt.title("RMSE vs PCA Values across models")
    plt.xlabel("PCA Values")
    plt.ylabel("RMSE")
    plt.grid()
    plt.plot(ada_data['pca'], ada_data['rmse'], marker='o', color='r', label="Adaboost")
    plt.plot(xg_data['pca'], xg_data['rmse'], marker='o', color='g', label="XgBoost")
    plt.plot(svr_data['pca'], svr_data['rmse'], marker='o', color='b', label="SVR")
    
    plt.legend(loc='best')
    plt.show()

In [ ]:
print()
print("========== PCA Tuning ==========")
print()

pca_ada_frame, pca_xg_frame, pca_svr_frame = pca_tuning([30,35,40,45,50,55,60,65,70], kaggle_train_X_processed, kaggle_train_y)

print()
print("*** PCA Tuning Complete ***")
print()

In [ ]:
plot_pca_tuning(pca_ada_frame, pca_xg_frame, pca_svr_frame)

# Learning Curves

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    plt.show()

In [ ]:
print()
print("========== Learning Curves ==========")
print()
plot_learning_curve(estimator=adaBoost, title="Learning Curves (AdaBoost)", X=kaggle_train_X_processed, y=kaggle_train_y.values.ravel(), ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5))
plot_learning_curve(estimator=xgb, title="Learning Curves (XgBoost)", X=kaggle_train_X_processed, y=kaggle_train_y.values.ravel(), ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5))
plot_learning_curve(estimator=svr_model, title="Learning Curves (SVR)", X=kaggle_train_X_processed, y=kaggle_train_y.values.ravel(), ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5))